## Импорт библиотек

In [1]:
import json
import re
import pandas as pd
import time 
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

## Памятка, что парсить

**Владивосток**: https://www.avito.ru/vladivostok/predlozheniya_uslug  
**Ростов**: https://www.avito.ru/rostov/predlozheniya_uslug  
**Санкт-Петербург**: https://www.avito.ru/sankt-peterburg/predlozheniya_uslug  
**Москва**: https://www.avito.ru/moskva/predlozheniya_uslug  

## Я обернул все функции в класс AvitoParser

In [2]:
class AvitoParser:
    
    #Конструктор класса, принимает ключ авито-api, 
    #selenium driver, ссылку api, 
    #а также категорию авито (в нашем случае это услуги)
    
    def __init__(self, key, driver, api_url, category_type_id):
        
        self.key = key
        self.driver = driver
        self.api_url = api_url
        self.category_type_id = category_type_id
    
    #Главная функция, принимает id города и аргумент parse_sub. 
    #Если parse_sub=False(по дефолту) - парсим толко главные категории, если True - парсим все до крайних.
    #На выходе отдаёт данные по городу в формате dataframe.
    
    def parse(self, location_id, parse_sub=False):
        
        final_data = []
        
        if location_id == '644560':
            city_name = 'Владивосток'
        elif location_id == '637640':
            city_name = 'Москва'   
        elif location_id == '662750':
            city_name = 'Ростов'
        elif location_id == '653240':
            city_name = 'Санкт-Петербург'
            
        main_body_pos = 1
        body_categories_pos = 15
        
        service_url = f'{self.api_url}11/items?key={self.key}&categoryId={self.category_type_id}&locationId={location_id}'
        self.driver.get(service_url)
        content = self.driver.page_source
        
        result = re.search('{(.*)}', content).group(0)
        json_data = json.loads(result)['result']
        categories_data = json_data['items'][main_body_pos]['value']['items'][body_categories_pos]['actions']
        
        for category in categories_data:
            
            if parse_sub:
                
                category_data = self.get_all_categories(category)
                
            else:
                
                category_data = self.get_main_categories(category)
                
            final_data.append(category_data)
            time.sleep(random.randint(1,5))
            
        df = pd.concat(final_data, ignore_index=True) 
        df['city_name'] = len(df)*[city_name]
        
        return df
    
    #Функция парсит данные главной категории, полученной на вход.
    #На выходе отдаёт данные по главной в формате dataframe.
    
    def get_main_categories(self, category):
        
        main_category = {}
        
        main_category['main_category_title'] = category['title']
        main_category['sub_category_title'] = None
        main_category_link = re.search('category(.*)', category['uri']).group(0)
        
        main_category_url = f"{self.api_url}11/items?key={self.key}&{main_category_link}"
        self.driver.get(main_category_url)
        content = self.driver.page_source
        
        result = json.loads(re.search('{(.*)}', content).group(0))
        main_category['sub_category_total'] = result['result']['totalCount']-result['result']["expanded_count"]
        
        time.sleep(random.randint(1,7))
        
        return pd.DataFrame([main_category])
    
    #Функция парсит данные подкатегорий категории, полученной на вход.
    #На выходе отдаёт данные по подкатегориям в формате dataframe.
    
    def get_all_categories(self, category):
    
        sub_categories_data = []

        if 'actions' not in category.keys():

            return self.get_main_categories(category)

        for sub in category['actions']:

            sub_categories = {}

            if sub['title']!='Все категории':

                sub_categories['main_category_title'] = category['title']

                if 'title' in sub.keys():

                    sub_categories['sub_category_title'] = sub['title']
                    sub_category_link = re.search('category(.*)', sub['uri']).group(0)

                    sub_category_url = f"{self.api_url}11/items?key={self.key}&{sub_category_link}"
                    self.driver.get(sub_category_url)
                    content = self.driver.page_source

                    result = json.loads(re.search('{(.*)}', content).group(0))
                    sub_categories['sub_category_total'] = result['result']['totalCount'] - result['result']["expanded_count"]
                    sub_categories_data.append(sub_categories)

                    time.sleep(random.randint(1,7))

        return pd.DataFrame(sub_categories_data)

## Настраиваем selenium, я использую Chrome webdriver

In [3]:
options = Options()
options.add_argument("--headless")
service=Service('chromedriver.exe')

driver = webdriver.Chrome(service=service, options=options)

## Создаём переменные, которые необходимо передать в наш класс и создаём экземпляр parser.

In [4]:
api_url = 'https://m.avito.ru/api/'
key = 'af0deccbgcgidddjgnvljitntccdduijhdinfgjgfjir'
category_type_id = '114'
location_id_vl = '644560'
location_id_msk = '637640'
location_id_rs = '662750'
location_id_spb = '653240'

parser = AvitoParser(key, driver, api_url, category_type_id)

## Вызываем метод parse нашего экземпляра и парсим каждый город

In [5]:
%%time

msk_df = parser.parse(location_id_msk)
time.sleep(random.randint(15,30))

rs_df = parser.parse(location_id_rs)
time.sleep(random.randint(15,30))

spb_df = parser.parse(location_id_spb)
time.sleep(random.randint(15,30))

vl_df = parser.parse(location_id_vl, parse_sub=True)

Wall time: 17min 18s


## Соединяем всё в единый dataframe и записываем в csv

In [6]:
all_df = pd.concat([msk_df, rs_df, spb_df, vl_df], ignore_index=True)
all_df.head()

,main_category_title,sub_category_title,sub_category_total,city_name
0,"Транспорт, перевозки",None,37111,Москва
1,"Ремонт, строительство",None,14443,Москва
2,Мастер на час,None,2538,Москва
3,"Сад, благоустройство",None,1800,Москва
4,"Красота, здоровье",None,28518,Москва


In [7]:
all_df.to_csv('parsed_categories.csv', index=False)

## Ссылка на API для получения номера телефона по идентификатору объявления (id_obj - идентификатор объявления).

In [8]:
id_obj = '1542618863'
phone_numbers = f'{api_url}1/items/{id_obj}/phone?key={key}'